# Fine Tuning do Modelo GPT-4.1 

1. Crie uma conta na Plataforma de desenvolvimento da OpenAI.
2. Insira pelo menos U$5,00 de crédito.
3. Crie uma chave API da OpenAI
4. Crie um script em Python (com auxílio de LLMs) para checar a acurácia do modelo gpt-4.1-nano frente ao problema de classificação de requisitos de sistemas nas categorias "funcional" e "não-funcional". Utilize o arquivo "dataset.jsonl" para isso. O script deverá exibir ao final a quantidade de classificações corretas e a porcentagem do total. Se você nunca programou em Python, peça auxílio de um LLM para instalar o Python e outras dependências necessárias. 
5. Realize o Fine Tuning do modelo gpt-4.1-nano com arquivo "dataset-train.jsonl". Ao criar o job de treinamento, use como sufixo o e-mail que você utilizou para se cadastrar ao classroom (e.g. "aluno@ga.ita.br").
6. Crie um script em Python (com auxílio de LLMs) para transformar o conteúdo do arquivo "dataset-test2.xlsx" numa base de dados em JSONL.
7. Una as instâncias constantes no arquivo "dataset-test.jsonl" com as instâncias oriundas do arquivo "dataset-test2.xlsx" (que você passou para JSONL no passo 6) para criar uma base de dados de teste com um número maior de instâncias. 
6. Crie outro script em Python (com auxílio de LLMs) para checar a acurácia do modelo gpt-4.1-nano "tunado", utilizando o dataset de teste que você criou no passo 7. O script deverá exibir ao final a quantidade de classificações corretas e a porcentagem do total. 
7. Entregue: 

     a) Todos os scripts. Lembre-se de documentar bem o seu código!
     
     b) O identificador único do seu modelo "tunado", nomeado com um sufixo com seu e-mail.

### Importação de Bibliotecas necessárias ao projeto

In [ ]:

# Pandas - biblioteca para manipulação e análise de dados estruturados
import pandas as pd

# JSON - biblioteca nativa do Python para trabalhar com dados em formato JSON
import json

# OS - biblioteca nativa para interação com o sistema operacional (caminhos, variáveis de ambiente, etc.)
import os

# OpenAI - cliente oficial da API da OpenAI para integração com modelos de linguagem
from openai import OpenAI

# python-dotenv - biblioteca para carregar variáveis de ambiente de arquivos .env
from dotenv import load_dotenv

### Transformação do conteúdo do arquivo "dataset-test2.xlsx" numa base de dados em JSONL. (Item 6)

In [ ]:
# =============================================================================
# CONFIGURAÇÃO DE CAMINHOS E CARREGAMENTO DE DADOS
# =============================================================================

# Define os caminhos de entrada (Excel) e saída (JSONL)
input_file = "data/xlsx/dataset-test2.xlsx"   # Arquivo Excel com os dados originais
output_file = "data/jsonl/dataset-test2.jsonl"  # Arquivo JSONL de saída para treinamento

# Carrega o dataset do arquivo Excel em um DataFrame do pandas
df = pd.read_excel(input_file)

# =============================================================================
# DEFINIÇÃO DO PROMPT DO SISTEMA
# =============================================================================

# Prompt que define o comportamento do modelo: classificar requirements como
# funcionais ou não-funcionais, retornando apenas uma das duas categorias
system_prompt = (
    "You are a Software Engineer and need to categorize requirements into "
    "'functional' or 'non-functional'. Your answer must be 'functional' or 'non-functional' only."
)

# =============================================================================
# CONVERSÃO PARA FORMATO JSONL (OPENAI FINE-TUNING)
# =============================================================================

# Abre o arquivo de saída em modo escrita com encoding UTF-8
with open(output_file, "w", encoding="utf-8") as f:
    # Itera sobre cada linha do DataFrame
    for _, row in df.iterrows():
        # Extrai e limpa o texto do requirement
        requirement = str(row["requirement"]).strip()
       
        # Interpreta a coluna "NF" para determinar o rótulo correto
        nf_value = row["NF"]
        if str(nf_value).strip().lower() in ["1", "non-functional", "nf"]:
            label = "non-functional"  # Requirement não-funcional
        else:
            label = "functional"      # Requirement funcional (padrão)
       
        # Cria o exemplo no formato esperado pela API OpenAI para fine-tuning
        # Estrutura: system prompt + entrada do usuário + resposta esperada
        example = {
            "messages": [
                {"role": "system", "content": system_prompt},      # Instruções do sistema
                {"role": "user", "content": requirement},          # Requirement a ser classificado
                {"role": "assistant", "content": label}            # Classificação esperada
            ]
        }
       
        # Escreve o exemplo como uma linha JSON no arquivo JSONL
        f.write(json.dumps(example, ensure_ascii=False) + "\n")

# Confirma a criação bem-sucedida do arquivo
print(f"✅ Arquivo JSONL criado em: {output_file}")

✅ Arquivo JSONL criado em: data/jsonl/dataset-test2.jsonl


### Criação do arquivo "dataset-test-final.jsonl", oriundo da união da instâncias do arquivo "dataset-test.jsonl" com as instâncias do arquivo "dataset-test2.xlsx". (Item 7) 

In [ ]:
# =============================================================================
# CONCATENAÇÃO DE DATASETS JSONL
# =============================================================================

# Define os arquivos de entrada que serão combinados
file1 = "data/jsonl/dataset-test2.jsonl"  # Primeiro dataset JSONL
file2 = "data/jsonl/dataset-test.jsonl"   # Segundo dataset JSONL

# Define o arquivo de saída que conterá os dados combinados
output_file = "data/jsonl/dataset-test-final.jsonl"

# =============================================================================
# PROCESSO DE CONCATENAÇÃO
# =============================================================================

# Abre o arquivo de saída em modo escrita
with open(output_file, "w", encoding="utf-8") as outfile:
    
    # Copia todas as linhas do primeiro arquivo
    with open(file1, "r", encoding="utf-8") as infile1:
        for line in infile1:
            outfile.write(line)  # Preserva a formatação JSONL original
    
    # Copia todas as linhas do segundo arquivo
    with open(file2, "r", encoding="utf-8") as infile2:
        for line in infile2:
            outfile.write(line)  # Adiciona ao final do primeiro dataset

# Confirma a operação bem-sucedida
print(f"✅ Arquivo combinado salvo em: {output_file}")

✅ Arquivo combinado salvo em: data/jsonl/dataset-test-final.jsonl


### Checando acurácia dos modelos. (Itens 4 e 8)

In [ ]:
# =============================================================================
# CONFIGURAÇÃO E INICIALIZAÇÃO
# =============================================================================

# Carrega as variáveis de ambiente do arquivo .env (incluindo chave da API)
load_dotenv()

# Inicializa o cliente da OpenAI com a chave de API do ambiente
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Define o arquivo de dataset que será usado para avaliação
dataset_file = "data/jsonl/dataset-test-final.jsonl"

# =============================================================================
# DEFINIÇÃO DOS MODELOS A COMPARAR
# =============================================================================

# Dicionário com os modelos que serão avaliados:
# - base: modelo padrão da OpenAI sem fine-tuning
# - fine_tuned: modelo personalizado treinado especificamente para esta tarefa
models = {
    "base": "gpt-4.1-nano-2025-04-14",  # Modelo base
    "fine_tuned": "ft:gpt-4.1-nano-2025-04-14:personal:te-251-carlos-figueiredo:CAUgUYUQ"  # Modelo fine-tuned
}

# =============================================================================
# INICIALIZAÇÃO DOS CONTADORES
# =============================================================================

# Cria estrutura para armazenar resultados de cada modelo
# Para cada modelo: total de exemplos testados e número de acertos
results = {name: {"total": 0, "correct": 0} for name in models}

# =============================================================================
# AVALIAÇÃO DOS MODELOS
# =============================================================================

# Processa cada exemplo do dataset de teste
with open(dataset_file, "r", encoding="utf-8") as f:
    for line in f:
        # Carrega o exemplo JSON
        example = json.loads(line)
        messages = example["messages"]
        
        # Extrai a resposta esperada (última mensagem do assistant)
        expected = messages[-1]["content"].strip().lower()
        
        # Cria mensagens para avaliação (remove a resposta correta)
        eval_messages = messages[:-1]  # System prompt + user input, sem o rótulo
        
        # Testa cada modelo com o mesmo exemplo
        for name, model in models.items():
            results[name]["total"] += 1  # Incrementa contador total
            
            # Faz a predição usando o modelo atual
            response = client.chat.completions.create(
                model=model,
                messages=eval_messages,
                temperature=0  # Determinístico para avaliação consistente
            )
            
            # Extrai e normaliza a resposta do modelo
            predicted = response.choices[0].message.content.strip().lower()
            
            # Verifica se a predição está correta
            if predicted == expected:
                results[name]["correct"] += 1  # Incrementa contador de acertos

# =============================================================================
# EXIBIÇÃO DOS RESULTADOS
# =============================================================================

# Calcula e exibe métricas de desempenho para cada modelo
for name, stats in results.items():
    total = stats["total"]
    correct = stats["correct"]
    
    # Calcula a acurácia (% de acertos)
    accuracy = (correct / total * 100) if total > 0 else 0
    
    # Exibe resultados formatados
    print(f"📌 Modelo: {models[name]}")
    print(f"   ✅ Total exemplos: {total}")
    print(f"   🎯 Acertos: {correct}")
    print(f"   📊 Acurácia: {accuracy:.2f}%\n")

📌 Modelo: gpt-4.1-nano-2025-04-14
   ✅ Total exemplos: 71
   🎯 Acertos: 40
   📊 Acurácia: 56.34%

📌 Modelo: ft:gpt-4.1-nano-2025-04-14:personal:te-251-carlos-figueiredo:CAUgUYUQ
   ✅ Total exemplos: 71
   🎯 Acertos: 58
   📊 Acurácia: 81.69%

